In [1]:
import pickle
from presentation_forms import table_repr, vertically

pentas = pickle.load(open('../../data/pentas.pickle', 'rb'))
table_repr([pentas, list(range(12))])

,,,,,,,,,,,
0,1,2,3,4,5,6,7,8,9,10,11


In [2]:
from z3 import *
from functools import reduce
from logic import Vocab, ptin


def switch_expr(expr, choices):
    return reduce(lambda rest, ichoice: If(expr == ichoice[0], ichoice[1], rest),
                  enumerate(choices), False)



class Placement:
    def __init__(self, penta, at, vari):
        self.penta = penta
        self.at = at
        self.vari = vari

    def translate(self, coords, shift):
        dx, dy = shift
        return [(x + dx, y + dy) for x, y in coords]

    def ptin(self, pt):
        vs = self.penta.variations()
        return switch_expr(self.vari,
                           [ptin(pt, self.translate(v.blocks, self.at)) for v in vs])

    def instantiate(self, at, vari):
        return Placement(self.penta.variations()[vari], at, 0)

    def from_model(self, m):
        return self.instantiate(tuple(m[self.at[i]].as_long() for i in [0,1]),
                                m[self.vari].as_long())


switch_expr(Int('x'), [5, 6, 7])

puz = [pentas[i] for i in [1,3,2,8,10,7,11,0]]
table_repr([puz])

,,,,,,,


In [3]:
vocab = Vocab()

placements = [Placement(penta, vocab.fresh_pt(), vocab.fresh('v')) for penta in puz]

s = Solver()
s.add(*[Or(p.ptin((x, y)) for p in placements) for x in range(len(puz)) for y in range(5)])
s.check()


sat

In [4]:
m = s.model()

#sol = [Placement(p.penta.variations()[m[p.vari].as_long()], 
#                 (m[p.at[0]].as_long(), m[p.at[1]].as_long()), 0) for p in placements]

sol = [p.from_model(m) for p in placements]

table_repr([[p.penta for p in sol],
            [p.at for p in sol]])

,,,,,,,
"(5, 0)","(2, 0)","(4, 3)","(1, 0)","(0, 0)","(0, 2)","(2, 3)","(5, 1)"


In [5]:
from presentation_forms import HTMLGrid

colors = ['#ff8', '#f88', '#cff', '#88f', '#fcf', '#4f8', '#539', '#f59']

HTMLGrid([(x, y, str(i)) for i, p in enumerate(sol) for x,y in p.translate(p.penta.blocks, p.at)],
        styles=[f'.htmlgrid-cell[data-txt="{i}"] {{ background: {c}; color: #0003; }}' for i, c in enumerate(colors)])